In [6]:
%matplotlib inline
import re
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib as mpl
from matplotlib.colors import SymLogNorm
import scienceplots

import astropy.units as u
from astropy import constants
from astropy.table import Table
from astropy.coordinates import SkyCoord,FK5,ICRS
from astropy.io import fits,ascii
from astropy.wcs import WCS, utils as wcsutils
from astropy.stats import sigma_clipped_stats
import astropy.visualization as vis
from astropy.modeling import models, fitting
from astropy.nddata import StdDevUncertainty, Cutout2D
from astropy.convolution import Gaussian2DKernel,Gaussian1DKernel
from astropy.cosmology import Planck18
from scipy.signal import find_peaks
#from scipy.constants import c



from photutils.aperture import EllipticalAperture, SkyEllipticalAperture

import os, sys, glob, pdb, scipy

from importlib import reload
import pyregion as pyreg
from spectral_cube import SpectralCube
from reproject import reproject_interp
import regions
from regions.shapes.circle import CircleSkyRegion,CirclePixelRegion
from regions import CircleAnnulusSkyRegion, CircleAnnulusPixelRegion, RectangleSkyRegion
from regions import PixCoord
sys.path.append('/disk/bifrost/yuanze/KBSS/MUSEQSO')
import run_cubetools_MUSE as ctools
brightu = r"Brightness $\rm (10^{-4}~erg~s^{-1}~arcsec^{-2}~cm^{-2}~\AA^{-1})$"
def imshow_astro(img, wcsinfo = None, figsize = (10,10), colorbar =True,
               cblabel="", cbfrac = 0.035, norm = None,
               stretch = vis.LinearStretch(), cmap = "hot",
               vrange = (None, None)):
    
    _, med, std = sigma_clipped_stats(img.data)
    
    fig =  plt.figure(figsize = figsize)
    if wcsinfo:
        ax = plt.subplot(projection = wcsinfo)
    else:
        ax = plt.subplot()
    
    vmin, vmax = vrange
    
    if not vmin:
        vmin = med
    if not vmax:
        vmax = med + 10*std
    if not norm:
        norm = vis.ImageNormalize(vmin = vmin, vmax = vmax, stretch = stretch)
    im = ax.imshow(img, norm = norm, cmap = cmap)
    if colorbar:
        cb = plt.colorbar(im, label = cblabel, fraction =cbfrac)
    return fig, ax, im
KBSSpath="/disk/bifrost/yuanze/KBSS"

#MUSEQSO= ascii.read(KBSSpath+"/KCWI/MUSEQSO_machine_readable_updated2.list",format="ipac")
#museqso_path = os.path.join(KBSSpath, "MUSEQSO")
#votable_file = os.path.join("fixed_file.xml")

In [9]:
from astropy.table import Table, Column
# Get a list of folders in the museqso_path
folders = [f for f in os.listdir(museqso_path) if os.path.isdir(os.path.join(museqso_path, f))]


# Create lists to store the 'found_in_folder' and 'file_count' information
found_in_folder = []
file_count = []

# Loop through the objects in the astropy_table to check folder presence and count files
for obj in MUSEQSO['Quasar']:
    if obj in folders:
        found_in_folder.append(True)
        folder_path = os.path.join(museqso_path, obj)
        num_files = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
        file_count.append(num_files)
    else:
        found_in_folder.append(False)
        file_count.append(0)

# Add the new columns to the table
MUSEQSO.add_column(Column(name='found_in_folder', data=found_in_folder))
MUSEQSO.add_column(Column(name='file_count', data=file_count))

# Print the updated table to check the new columns
print(MUSEQSO)

# Write the updated table back to a new VOTable file
#new_votable = from_table(astropy_table)
#new_votable_file = os.path.join(museqso_path, "ESO_export_updated.xml")
new_musetable_file=KBSSpath+"/MUSEQSO/MUSEQSO_machine_readable_updated.list"
#ascii.write(MUSEQSO, KBSSpath+"/MUSEQSO/MUSEQSO_machine_readable_updated.list",format="ipac", overwrite=True) 

print(f"Updated MUSEQSO master table written to {new_musetable_file}")

 ID       Quasar          RA      ... Class found_in_folder file_count
unit       unit          unit     ...  unit                           
---- --------------- ------------ ... ----- --------------- ----------
   1  SDSSJ2319-1040 23:19:34.800 ...    RQ            True          1
   2            UM24 00:15:27.400 ...    RQ            True          1
   3       J0525-233 05:25:06.500 ...    RL            True          1
   4      Q-0347-383 03:49:43.700 ...    RQ            True          2
   5  SDSSJ0817+1053 08:17:52.099 ...    RQ            True          1
   6  SDSSJ0947+1421 09:47:34.200 ...    RQ            True          1
   7  SDSSJ1209+1138 12:09:18.000 ...    RQ            True          1
   8           UM683 03:36:26.900 ...    RQ            True          1
   9      Q 0956+122 09:58:52.200 ...    RQ            True          1
  10 SDSSJ10251+0452 10:25:09.600 ...    RQ            True          1
 ...             ...          ... ...   ...             ...        ...
  71  

In [30]:
hdu0[1].header

XTENSION= 'IMAGE   '           / IMAGE extension                                
BITPIX  =                  -32 / number of bits per data pixel                  
NAXIS   =                    3 / number of data axes                            
NAXIS1  =                  353 / length of data axis 1                          
NAXIS2  =                  330 / length of data axis 2                          
NAXIS3  =                 3681 / length of data axis 3                          
PCOUNT  =                    0 / required keyword; must = 0                     
GCOUNT  =                    1 / required keyword; must = 1                     
EXTNAME = 'DATA    '           / This extension contains data values            
DATASUM = '1440636081'         / data unit checksum updated 2016-05-30T11:13:35 
HDUCLASS= 'ESO     '           / class name (ESO format)                        
HDUDOC  = 'DICD    '           / document with class description                
HDUVERS = 'DICD version 6'  

In [154]:
import makeMask_MUSE as mmask
reload(mmask)
from datetime import datetime# Making white-light cutout
root_directory = KBSSpath+"/MUSEQSO"
source_table = ascii.read(root_directory+"/MUSEQSO_machine_readable_updated2.list",format="ipac")#QSOtab=qsos[(qsos['contam']=="False")&(qsos['Field']!="Q1623")]
filters = ["table['file_count'] < 2", "table['M_i'] > -29.6"]
#QSOtab=qsos[(qsos['Field']!="Q0142")&(qsos['Field']!="Q1623")]
all_directories,tab = ctools.find_directories_from_ascii(source_table,root_directory,filters=filters)
print("Number of directories found:",len(all_directories))
#gc.set_debug(gc.DEBUG_LEAK)

for subdir in all_directories: 
    qname=os.path.basename(subdir)
    srow = source_table[source_table['Quasar'] == qname]
   # qname="LBQS 2139-4434"
    adp_file=ctools.find_adp_fits_file(subdir)
    hdu0=fits.open(f"{adp_file}.PSFSub.fits")
    hdr = hdu0[1].header
    wc = (np.arange(hdr['NAXIS3']) - hdr['CRPIX3'] + 1) * hdr['CD3_3'] + hdr['CRVAL3']
    #NSubfile = adp_prefix+".PSFSub.fits"
    #zmask = np.loadtxt(subdir+"/zmask.txt").astype(np.int64)
    zmask = mmask.determine_mask(wc,"continuum", srow['z_Lyaneb'])
    psfcen = np.loadtxt(subdir+"/psfcen.txt").astype(np.int64)
    xpix = psfcen[0,1]
    ypix = psfcen[0,2]
    wimage = np.nanmean(hdu0[1].data[zmask[0]:zmask[1],:],axis=0)
    
    header = fits.Header()
    header['DATE'] = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
    header['CREATOR'] = 'Yuanze Ding'
    header['COMMENT'] = 'White-light cutout from 3D data cube'
    header['YSOURCE'] = ypix
    header['XSOURCE'] = xpix
    header['OBJECT'] = qname
    # Extract WCS information from the original header
    wcs_keys = ['CTYPE1', 'CTYPE2', 'CRVAL1', 'CRVAL2', 'CRPIX1', 'CRPIX2', 'CDELT1', 'CDELT2', 'CUNIT1', 'CUNIT2', 'CD1_1', 'CD1_2', 'CD2_1', 'CD2_2']
    for key in wcs_keys:
        if key in hdr:
            header[key] = hdr[key]
            
    
    hdu = fits.PrimaryHDU(wimage,header=header)
    hdulist = fits.HDUList([hdu])
    output_fn = f"{adp_file}.white.fits"
    hdulist.writeto(output_fn, overwrite=True)
#hdu2=fits.open(KBSSpath+"/MUSEQSO/Q-0347-383/ADP.2016-06-02T13:21:57.103.fits")

Number of directories found: 63


/tmp/ipykernel_1220950/2807129781.py:26: RuntimeWarning: Mean of empty slice
  wimage = np.nanmean(hdu0[1].data[zmask[0]:zmask[1],:],axis=0)


In [9]:
sys.path.append('/disk/bifrost/yuanze/software/GalfitS/src')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
import images as IM
reload(IM)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


<module 'images' from '/disk/bifrost/yuanze/software/GalfitS/src/images.py'>

In [12]:
def write_fits_cube(data,header_ref, outputname):
    #wcs_keys = ["XSOURCE","YSOURCE","OBJECT",'CTYPE1',\
    #            'CTYPE2', 'CRVAL1','CRVAL2', 'CRPIX1',\
    #            'CRPIX2', 'CDELT1', 'CDELT2', 'CUNIT1',\
     #           'CUNIT2', 'CD1_1', 'CD1_2', 'CD2_1', 'CD2_2']
    #for key in wcs_keys:
        #if key in header:
            #header[key] = header_ref[key]
    primary = fits.PrimaryHDU(data, header=header_ref)
    #hdu_var = fits.ImageHDU(np.float32(variance), header=header)
    #hdu_temp = fits.ImageHDU(data, header=header)
#        hdu_temp2 = fits.ImageHDU(np.float32(self.cut_mask_image))
#        hdu_temp3 = fits.ImageHDU(np.float32(self.cut_sigma_image))
    hdul = fits.HDUList([primary])
#    print(hdul[0].header)
    hdul.writeto(outputname, overwrite=True)
    return 0
def maskplot(data,mask,output=None,center=None):
    from astropy.visualization.mpl_normalize import simple_norm
    sky_mean, sky_median, sky_std = sigma_clipped_stats(data, sigma=3, maxiters=5)
    norm = simple_norm([sky_std, 10*sky_std], 'linear', percent=99.5)
    fig, ax = plt.subplots(figsize=(3,3),dpi=300)
    ax.imshow(data-sky_median,origin='lower',cmap='gray_r',norm=norm)
    #plt.imshow(img.cut_sigma_image,origin='lower',cmap='gray_r')
    ax.imshow(mask,origin='lower',cmap='Blues',alpha=mask.astype(float)*0.7)
    if center is not None:
        ax.plot(center[1],center[0],"+",c="r",ms=10,lw=0.01)
    if output is not None:
        fig.savefig(output,dpi=300)
    plt.close()
    return 0
def create_circular_mask(shape, center, radius):
    # Create grid of coordinates
    Y, X = np.ogrid[:shape[0], :shape[1]]
    
    # Calculate distance from the center
    dist_from_center = np.sqrt((X - center[1])**2 + (Y - center[0])**2)
    
    # Create mask
    mask = dist_from_center <= radius
    return mask

In [14]:
root_directory = KBSSpath+"/MUSEQSO"
source_table = ascii.read(root_directory+"/MUSEQSO_machine_readable_updated2.list",format="ipac")#QSOtab=qsos[(qsos['contam']=="False")&(qsos['Field']!="Q1623")]
filters = ["table['file_count'] < 2", "table['M_i'] < -29.6"]
#QSOtab=qsos[(qsos['Field']!="Q0142")&(qsos['Field']!="Q1623")]
all_directories,tab = ctools.find_directories_from_ascii(source_table,root_directory,filters=filters)
print("Number of directories found:",len(all_directories))
#gc.set_debug(gc.DEBUG_LEAK)
cutr=30 # arcsec cut image size on each sides
for subdir in all_directories: 
    quasar_name=os.path.basename(subdir)
    source_row = source_table[source_table['Quasar'] == quasar_name]
    ra = source_row['RA'][0]
    dec = source_row['Decl'][0]
    sc= SkyCoord(ra,dec,unit=(u.hourangle,u.deg))
   # qname="LBQS 2139-4434"
    adp_file=ctools.find_adp_fits_file(subdir)
    fname=f"{adp_file}.white.fits"
    img = IM.image(fname,hdu=0,unit=u.Jy)
    psfcen = np.loadtxt(subdir+"/psfcen.txt").astype(np.int64)
    xpix = psfcen[0,1]
    ypix = psfcen[0,2]
    center = [ypix,xpix]
    imcut, cp = img.img_cut(sc.ra.value,sc.dec.value,cutr)
    img.sources_skycord_cut = [xpix,ypix]
    #img.cut_mask_image = np.zeros_like(img.cut_sigma_image)
    sdmask = create_circular_mask(img.data.shape, center, 17)
    img.cut_mask_image = img.generate_cutmask(6,nsigma=3,sdmask=sdmask,data = img.data.data,deblend=True,addgrow=1,nlevels=32,source_dia=20,contrast=0.01)
    
    maskplot(img.data.data,img.cut_mask_image,output=root_directory+"/diagnostic/"+quasar_name+"_maskmap.pdf",center=center)
    write_fits_cube(img.cut_mask_image.astype(np.int32),img.data.header,f"{adp_file}.mask.fits")
    #NSubfile = adp_prefix+".PSFSub.fits"
    #zmask = np.loadtxt(subdir+"/zmask.txt",dtype=int)
    
    
    

Number of directories found: 15
3.8641666666666663 6.67
151.677359535054 159.70674938232614


146.89249999999996 14.354722222222222
159.34517664849125 156.9720176185852


149.71749999999997 12.045833333333333
184.96413254851058 186.8192626002706


196.7925 12.505833333333333
173.60206471018805 139.0421391293419


331.8929166666666 -40.615833333333335
171.1117040451304 147.2649622175369


...

168.4608333333333 -15.559166666666668
151.77854864670837 199.75773192025994


200.12416666666664 -5.392833333333334
187.93167674090714 168.3865110662362


245.3195833333333 -0.7133888888888889
201.73852898327644 182.55113731534377


294.9891666666666 -10.044416666666667
175.70716572971835 174.46204698019702


300.85041666666666 -32.86275
222.2995955159318 220.2122899907279


In [11]:
img.data.header

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -32 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                  318                                                  
NAXIS2  =                  315                                                  
DATE    = '2024-06-28T00:23:27'                                                 
CREATOR = 'Yuanze Ding'                                                         
YSOURCE =                  158                                                  
XSOURCE =                  161                                                  
OBJECT  = 'SDSSJ2319-1040'                                                      
CTYPE1  = 'RA---TAN'                                                            
CTYPE2  = 'DEC--TAN'                                                            
CRVAL1  =           349.8950

In [125]:
img.sources_skycord_cut

[216, 226]

In [126]:
all_directories

['/disk/bifrost/yuanze/KBSS/MUSEQSO/UM24',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/SDSSJ0947+1421',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/Q 0956+122',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/SDSSJ1307+1230',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/Q-2204-408',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/Q2355+0108',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/CTS_G18.01',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/Q0055-269',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/CT_656',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/HE0940-1050',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/CTS_R07.04',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/Q1317-0507',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/QSO J1621-0042',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/PKS1937-101',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/QB2000-330']

In [140]:
fitsmask=fits.open(f"{adp_file}.mask.fits")[0].data

In [ ]:
mask_3d=np.zeros((3631,449,454),dtype=bool)
for i in range(mask_3d.shape[0]):
    mask_3d[i,:,:]=np.bitwise_or(mask_3d[i,:,:],fitsmask)

In [152]:
from astropy import cosmology

In [153]:
all_directories

['/disk/bifrost/yuanze/KBSS/MUSEQSO/UM24',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/SDSSJ0947+1421',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/Q 0956+122',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/SDSSJ1307+1230',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/Q-2204-408',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/Q2355+0108',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/CTS_G18.01',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/Q0055-269',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/CT_656',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/HE0940-1050',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/CTS_R07.04',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/Q1317-0507',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/QSO J1621-0042',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/PKS1937-101',
 '/disk/bifrost/yuanze/KBSS/MUSEQSO/QB2000-330']